## 1. Extract data from the existing database

In [1]:
!pip install openai 
!pip install pandas
!pip install os
!pip install sqlalchemy
!pip install tqdm
!pip install string
!pip install json
!pip install python-dotenv
!pip install neo4j
!pip install timeit
!pip install time
!pip install nest_asyncio
!pip install sql
!pip3 install ipython-sql
!pip install jupysql pandas pyarrow --quiet
!pip install psycopg2-binary --quiet


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
ERROR: Could not find a version that satisfies the requirement string (from versions: none)
ERROR: No matching distribution found for string
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
ERROR: Could not find a version that satisfies the requirement timeit (from versions: none)
ERROR: No matching distribution found for timeit
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [1]:

## Load packages

import asyncio
import pandas as pd
import os
import openai
from openai import OpenAI
from dotenv import load_dotenv
from sqlalchemy import create_engine
from tqdm import tqdm
from string import Template
import json 
from neo4j import GraphDatabase
from timeit import default_timer as timer 
from time import sleep 
#import nest_asyncio
#import sql
import psycopg2
from pprint import pprint
import igraph as ig
import leidenalg as la
import re

In [2]:
## OpenAI API credentials
load_dotenv()

openai_api_key= os.getenv('OPENAI_API_KEY')

## Neo4j credentials

neo4j_url = os.getenv("NEO4J_URI")
neo4j_user = os.getenv("NEO4J_USERNAME")
neo4j_password = os.getenv("NEO4J_PASSWORD")
gds = GraphDatabase.driver(neo4j_url, auth = (neo4j_user, neo4j_password))

In [3]:
## Helper Functions
def process_gpt(file_prompt, system_msg):
    client = OpenAI(api_key= openai_api_key)

    completion = client.chat.completions.create(
        model = 'gpt-4o-mini',
        max_tokens = 15000,
        temperature = 0, 
        messages = [
            {"role": "system", "content": "Please output valid JSON"},
            {"role":"user", "content": file_prompt}
        ]
    )
    nlp_results = completion.choices[0].message.content
    nlp_results = nlp_results.replace('\n','')
    nlp_results = nlp_results.replace('\\n','')
    nlp_results = nlp_results.replace('\\\\','')
    reply = json.loads(nlp_results)
    return nlp_results

## Function to take folder of fieles and a prompt template, and return a json-object of all the entities and relationships 

def extract_entities_relationships(content, prompt_template):
    start = timer()
    system_msg = "You are a helpful IT-project and account management expert who extracts information from documents."
    print(f"Running pipeline for {len(content)} files")
    results = []
    for i in range(len(content)):
        try:
            print(f"Extracting entities and relationships for file number: {i}")
            text = content[i]
            prompt = Template(prompt_template).substitute(ctext=text)
            result = process_gpt(prompt, system_msg=system_msg)
            results.append(json.loads(result))
        except Exception as e:
            # Log the error and continue with the next item
            print(f"Error processing file number {i}: {e}")
            # Optionally, you can append a placeholder or `None` to maintain the list's structure
            results.append(None)  # or append some other placeholder value
    end = timer()
    print(f"Pipeline completed in {end-start} seconds")
    return results

    
def add_doc_id_to_entities(results, doc_ids):
    for i, result in enumerate(results):
        if result is None:  # Skip None values
            continue
        doc_id = doc_ids[i]   
        entities = result.get("entities", [])
        for entity in entities:
            entity["doc_id"] = doc_id
    return results

In [4]:
df = pd.read_csv("files/final_df_2.csv")
df

,Unnamed: 0,doc_id,content
0,0,00019fdfa72ca963b5d236470f52e2e4,Q and A with David Stainforth on Predicting Ou...
1,1,0003a625eaa9f248e1467532ced367b7,UK General Election 2024 at LSE - YouTubeAbout...
2,2,001a5f6677681e52d4bf6d29614b2983,Other wellbeing resources\rBrowser does not su...
3,3,001a68fb3453d90cb36daa76f0f3da19,Contacting the Student Services Centre\rBrowse...
4,4,0022b72249c01371d18c5ac97509b584,REF 2014\rBrowser does not support script.Skip...
...,...,...,...
4101,4091,ffaed91a507cd6e8f0653717a00a3cc0,Publications of our PhDs\rBrowser does not sup...
4102,4092,ffb07eb5be71d17c879d599b9e891794,LSE apprentices involved in design of new publ...
4103,4093,ffdb1303e620c76b2314476e747c1e5c,1 Revision History – this document was amended...
4104,4094,ffe37e4d77ae4078591f663a0f690547,Dr. Alex Mayhew\rBrowser does not support scri...


## 2. Prompt engineering for entity and relations extraction 

In [5]:
#department_process_prompt is to handle academics-related entities and relationships

department_process_prompt = '''From the unstructured data provided, extract the following Entities and relationships described in the mentioned format.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses.
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
    'id' property of each enetity must be alphanumeric and must be unique among the entities. You will be referring to this property to define the relationships between entities.
    Entity types:
    label: 'Department', id:string, name:string // Academic departments at the London School of Economics, referred to as "Department of ____"; 'id' property is the name of the department, in lowercase and camel-case.
    label: 'Course', id:string, name:string, course_code:string, prerequisites:string, semester:string // Academic courses taught at the London School of Economics; 'id' property is the name of the course, in lowercase and camel-case
    label: 'Faculty', id:string, name:string // Professors, assistant professors, teachers, fellows, tutors, directors, lecturers, and chairs associated with the London School of Economics; 'id' property is the full-name of the faculty member, in lowercase and camel-case
    label: 'Event', id:string, name:string, description:string, date:string // Seminars, workshops, talks, and other events involving the London School of Economics; 'id' property is the name of the event, in lowercase and camel-case
    label: 'Research Project', id:string, name:string, description:string // Research projects led by faculty associated with the London School of Economics; 'id' property is the name of the research project, in lowercase and camel_case


2. Next, generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective 'id' property.
    Relationship types:
    faculty[IS_MEMBER_OF]department
    event[IS_HOSTED_BY]faculty
    event[IS_RUN_BY]department
    course[IS_RUN_BY]department
    course[IS_TAUGHT_BY]faculty
    research_project[IS_LED_BY]faculty
    research_projects[IS_ASSOCIATED_WITH]department

The output should look like:
{
    "entities": [{"label": "Department", "id": string, "name": string}],
    "relationships": ["faculty[IS_MEMBER_OF]department", "event[IS_RUN_BY]department", "course[IS_RUN_BY]department", "research_projects[IS_ASSOCIATED_WITH]department"]
}

Case Sheet:
$ctext
'''

#administration_process_prompt is to handle bureaucratic and regulatory entities and relationships

administration_process_prompt = '''From the unstructured data provided, extract the following Entities and relationships described in the mentioned format.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses.
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
    'id' property of each entity must be alphanumeric and must be unique among the entities. You will be referring to this property to define the relationships between entities.
    Entity types:
    label: "Policy", id: string, name:string, policy_procedure_summary:string // Policy related to administrative procedure at the London School of Economics; 'id' property is the name of the policy, in lowercase and camel_case
    label: "Committee", id:string, name: string // Regulatory organisation or departments at the London School of Economics, referred to as "Department of ____"; 'id' propoerty is the name of the committee, in lowercase and camel-case
    label: "Group", id:string, name:string // Individuals or organisations; 'id' property is the name of the perosn, in lowercase and camel_case

2. Next, generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective 'id' property.
    Relationship types:
    group[IS_AFFECTED_BY]policy
    policy[IS_ENFORCED_BY]committee
    
The output should look like:
{ "entities": [{"label": "Policy", "id": string, "name": string}],
    "relationships": ["group[IS_AFFECTED_BY]policy"]
}

Case Sheet: $ctext
''' 

#institutions_process_prompt is to handle institutions at the LSE (research facilities, summer school, careers centre, alumni centre)

#institutions_process_prompt = 
#research_process_prompt = 

full_process_prompt = '''From the unstructured data provided, extract the following Entities and relationships described in the mentioned format.
0. ALWAYS FINISH THE OUTPUT. Never send partial responses. If the input text is too long to process in one response, prioritize completing entities and relationships in the output rather than sending incomplete data.
1. First, look for these Entity types in the text and generate as comma-separated format similar to entity type.
    'id' property of each entity (EXCEPT FOR 'Document' ENTITY) must be alphanumeric and must be unique among the entities. You will be referring to this property to define the relationships between entities.
    Entity types:
    label: 'Department', id:string, name:string, description:string, head:string, website:string, other_relevant_content:string // Academic departments at the London School of Economics, referred to as "Department of ____"; 'id' property is the name of the department, in lowercase and camelCase.
    label: 'Degree': id:string, name:string, description:string, duration:string, level:string, credits_required:string, website:string, other_relevant_content:string // Bachelor or masters university degrees that are taken at the London School of Economics, that includes "BSc Mathematics and Economics", "BA Geography", "BSc Data Science"; 'id' property is the name of the degree, in lowercase and camelCase.
    label: 'Module', id:string, name:string, course_code:string, prerequisites:string, semester:string, course_content_summary:string, description:string, credit_value:string, assessment_methods:string, learning_outcomes:string, recommended_readings:string, other_relevant_content:string // Academic courses taught at the London School of Economics; 'id' property is the name of the course, in lowercase and camelCase.
    label: 'Faculty', id:string, name:string, description:string, position:string, email:string, phone_number:string, office_hours:string, other_relevant_content:string // Professors, assistant professors, teachers, fellows, tutors, directors, lecturers, and chairs associated with the London School of Economics; 'id' property is the full name of the faculty member, in lowercase and camelCase.
    label: 'Event', id:string, name:string, description:string, date:string, location:string, organizer:string, registration_link:string, contact_person:string, other_relevant_content:string // Seminars, workshops, talks, and other events involving the London School of Economics; 'id' property is the name of the event, in lowercase and camelCase.
    label: 'ResearchProject', id:string, title:string, summary:string, description:string, start_date:string, end_date:string, principal_investigator:string, funding_amount:string, other_relevant_content:string // Research projects led by faculty associated with the London School of Economics; 'id' property is the title of the research project, in lowercase and camelCase.
    label: "Policy", id:string, name:string, policy_procedure_summary:string, description:string, effective_date:string, last_reviewed_date:string, contact_person:string, related_policies:string, other_relevant_content:string // Policy related to administrative procedure at the London School of Economics; 'id' property is the name of the policy in lowercase and camelCase.
    label: "AdministrativeCommittee", id:string, name:string, description:string, chair:string, meeting_frequency:string, members:string, contact_email:string, other_relevant_content:string // Regulatory organisation or departments at the London School of Economics; 'id' property is the name of the committee in lowercase and camelCase.
    label: "Group", id:string, name:string, description:string, membership_criteria:string, number_of_members:string, leader:string, affiliated_societies:string, other_relevant_content:string // Individuals or organisations that are DIRECTLY AFFECTED BY ADMINISTRATIVE POLICIES AND PROMPTS at the London School of Economics; 'id' property is the name of the group in lowercase and camelCase.
    label: 'Publication', id:string, title:string, summary:string, description:string, publication_date:string, journal:string, doi:string, authors:string, other_relevant_content:string // A publication resulting from research conducted at LSE; 'id' property is the title of the publication in lowercase and camelCase.
    label: 'FundingSource', id:string, name:string, description:string, contact_person:string, funding_amount:string, website:string, other_relevant_content:string // An entity providing funding for research projects; 'id' property is the name of the funding source in lowercase and camelCase.
    label: 'Society', id:string, name:string, type:string, description:string, founded_year:string, current_members:string, membership_criteria:string, website:string, other_relevant_content:string // Clubs, sports clubs, and other student societies at the London School of Economics; 'id' property is the name of the society in lowercase and camelCase.
    label: 'HeadOfSociety', id:string, name:string, description:string, contact_email:string, term_start:string, term_end:string, other_relevant_content:string // Individuals leading a society, such as presidents or captains; 'id' property is the name of the individual in lowercase and camelCase.
    label: 'Facility', id:string, name:string, location:string, description:string, capacity:string, operating_hours:string, manager:string, contact_number:string, other_relevant_content:string // Facilities offered by the Student Union, such as the gym, faith center, performance venues, etc.; 'id' property is the name of the facility in lowercase and camelCase.
    label: 'Support', id:string, name:string, description:string, eligibility_criteria:string, contact_email:string, availability:string, location:string, other_relevant_content:string // Support services provided by the Student Union, such as counseling, financial aid, or advisory services; 'id' property is the name of the support service in lowercase and camelCase.
    label: 'Building', id:string, name:string, type:string, address:string, description:string, construction_year:string, floors:string, main_occupants:string, accessibility_features:string, other_relevant_content:string // Academic buildings at the London School of Economics; 'id' property is the name of the building in lowercase and camelCase.
    label: 'Office', id:string, name:string, location:string, description:string, head:string, phone_number:string, email_address:string, office_hours:string, other_relevant_content:string // Offices or administrative units within buildings; 'id' property is the name of the office in lowercase and camelCase.
    label: 'Institution', id:string, name:string, description:string, founding_year:string, director:string, affiliated_departments:string, website:string, other_relevant_content:string // Institutions affiliated with the London School of Economics that are not academic departments, including "Data Science Institute", "Eden Centre", "LSE Summer School"; 'id' property is the name of the institution in lowercase and camelCase.

    Note: If multiple instances of an entity have similar names, append a numeric suffix, contextual attribute, timestamp, or other distinguishing features to the 'id' property to ensure uniqueness. For example, "john_smith" and "john_smith_1", "common_room_economics" and "common_room_management", "research_project_2023" and "research_project_2024".
    Note: For the "other_relevant_information" attribute, include ALL information about the entity that is not already included in other attributes. 

2. Next, generate each relationship as triples of head, relationship, and tail. To refer to the head and tail entity, use their respective 'id' property.
    Relationship types:
    faculty[IS_MEMBER_OF]department
    event[IS_HOSTED_BY]faculty
    event[IS_RUN_BY]department
    module[IS_RUN_BY]department
    module[IS_TAUGHT_BY]faculty
    research_project[IS_LED_BY]faculty
    research_project[IS_ASSOCIATED_WITH]department
    degree[IS_OFFERED_BY]department
    module[IS_STUDIED_IN]degree
    module[IS_PREREQUISITE_FOR]module
    faculty[IS_IN_CHARGE_OF]module
    faculty[IS_IN_CHARGE_OF]degree
    faculty[RESIDES_IN]office
    group[IS_AFFECTED_BY]policy
    policy[IS_ENFORCED_BY]administrative_committee
    researcher[CONDUCTS]research_project
    research_project[IS_ASSOCIATED_WITH]department
    research_project[RESULTS_IN]publication
    research_project[IS_FUNDED_BY]funding_source
    publication[IS_AUTHORED_BY]researcher
    research_project[IS_GOVERNED_BY]policy
    head_of_society[LEADS]society
    society[USES]facility
    event[IS_HOSTED_BY]society
    event[TAKES_PLACE_AT]facility
    support[IS_PROVIDED_BY]facility
    department[IS_LOCATED_IN]building
    office[IS_LOCATED_IN]building
    facility[IS_PART_OF]building
    institution[IS_AFFILIATED_WITH]institution


NOTE: For the relationships template above, replace each entity label with the id of the entity in the relationship. For example, an instance of "module[IS_TAUGHT_BY]faculty" is "ds105[IS_TAUGHT_BY]profJonCardosoSilva". DO NOT write the label of the entity in the relationship.

NOTE: If a required entity or relationship is not present in the text, do not attempt to generate or infer it. Only extract and include entities and relationships that are explicitly present and verifiable in the document.

NOTE: Ensure all generated output is in a single, well-formed JSON object. Avoid any trailing commas or syntax errors. Do not include instances of "\n". The output should be a single JSON object for each document, with "entities" and "relationships" as keys.

The output MUST be a dictionary for EACH document that you process, with each key being "entities" and "relationships". Here is a full example:
{
  "entities": [
    {
      "label": "Document",
      "id": "1",
      "title": "Annual Report 2023",
      "type": "Report",
      "description": "Comprehensive review of activities and outcomes for the year 2023",
      "date": "2024-01-01",
      "author": "John Doe",
      "other_relevant_content": "https://www.lse.ac.uk/reports/annual2023"
    },
    {
      "label": "Policy",
      "id": "examProceduresForCandidates",
      "name": "Exam Procedures for Candidates",
      "policy_procedure_summary": "Procedures and guidelines that candidates must follow during exam periods to avoid allegations of misconduct and ensure the proper conduct of exams.",
      "description": "A policy detailing the conduct expected from candidates during exams.",
      "effective_date": "2023-09-01",
      "last_reviewed_date": "2024-01-15",
      "contact_person": "policyAdminTeam",
      "related_policies": "academicIntegrityPolicy",
      "other_relevant_content": "https://www.lse.ac.uk/policies/exam-procedures"
    },
    {
      "label": "Support",
      "id": "studentServicesCentre",
      "name": "Student Services Centre",
      "description": "Central hub for student services including registration, counseling, and support.",
      "eligibility_criteria": "All enrolled students",
      "contact_email": "ssc@lse.ac.uk",
      "availability": "Monday to Friday, 9am - 5pm",
      "location": "Old Building, Room G12",
      "other_relevant_content": "https://www.lse.ac.uk/student-services"
    },
    {
      "label": "AdministrativeCommittee",
      "id": "examsTeam",
      "name": "Exams Team",
      "description": "Committee responsible for the administration and oversight of exams.",
      "chair": "profJohnSmith",
      "meeting_frequency": "Monthly",
      "members": "profJaneDoe, profAlanWhite",
      "contact_email": "examsteam@lse.ac.uk",
      "other_relevant_content": "https://www.lse.ac.uk/exams"
    },
    {
      "label": "AdministrativeCommittee",
      "id": "studentRegulationsTeam",
      "name": "Student Regulations Team",
      "description": "Committee overseeing student conduct and regulatory compliance.",
      "chair": "profEmilyClark",
      "meeting_frequency": "Bi-weekly",
      "members": "profMichaelBrown, profSaraGreen",
      "contact_email": "regulations@lse.ac.uk",
      "other_relevant_content": "https://www.lse.ac.uk/regulations"
    },
    {
      "label": "Support",
      "id": "disabilityAndMentalHealthService",
      "name": "Disability and Mental Health Service",
      "description": "Provides support for students with disabilities and mental health needs.",
      "eligibility_criteria": "Students with registered disabilities or mental health needs",
      "contact_email": "dmh@lse.ac.uk",
      "availability": "Monday to Friday, 9am - 5pm",
      "location": "East Building, Room 3.01",
      "other_relevant_content": "https://www.lse.ac.uk/disability-mental-health"
    }
  ],
  "relationships": [
    "1[INCLUDES]examProceduresForCandidates",
    "1[INCLUDES]studentServicesCentre",
    "1[INCLUDES]examsTeam",
    "1[INCLUDES]studentRegulationsTeam",
    "1[INCLUDES]disabilityAndMentalHealthService",
    "studentServicesCentre[IS_ASSOCIATED_WITH]examProceduresForCandidates",
    "examsTeam[IS_ASSOCIATED_WITH]examProceduresForCandidates",
    "studentRegulationsTeam[IS_ASSOCIATED_WITH]examProceduresForCandidates",
    "disabilityAndMentalHealthService[IS_ASSOCIATED_WITH]examProceduresForCandidates"
  ]
}

NOTE: Do not include instances of "\n".

Case Sheet:
$ctext
'''


In [7]:
df_content = df["content"].to_list()

result = extract_entities_relationships(df_content, full_process_prompt)

with open("files/entities_and_relations_test.json", "w") as f:
  json.dump(result, f, indent = 4)

Running pipeline for 5 files
Extracting entities and relationships for file number: 0
Error processing file number 0: 0
Extracting entities and relationships for file number: 1
Error processing file number 1: 1
Extracting entities and relationships for file number: 2
Error processing file number 2: 2
Extracting entities and relationships for file number: 3
Error processing file number 3: 3
Extracting entities and relationships for file number: 4
Error processing file number 4: 4
Pipeline completed in 0.0007063099183142185 seconds


In [13]:
with open("files/entities_and_relations_2.json", "r") as file:
    text = json.load(file)

doc_ids = df["doc_id"].to_list()
results_with_doc_id = add_doc_id_to_entities(text, doc_ids)

with open("files/entities_and_relations_2.json", "w") as f:
    json.dump(results_with_doc_id, f, indent=4)

## 3. Neo4j Cypher generation using Leiden-algorithm generated community levels

In [ ]:
import json
import igraph as ig
from leidenalg import find_partition, ModularityVertexPartition

# Load the JSON data
with open('files/entities_and_relations_2.json', 'r') as file:
    data = json.load(file)

# Extract entities and relationships
entities = {}
edges = []

for graph in data:
    # Create nodes for each entity
    if graph:
        for entity in graph['entities']:
            if entity:
                entities[entity['id']] = entity['label']
            else:
                continue

# Create an igraph Graph
g = ig.Graph()
g.add_vertices(list(entities.keys()))
g.add_edges(edges)

# Apply the Leiden algorithm
partition = find_partition(g, ModularityVertexPartition)

# Assign community labels to nodes
community_labels = {g.vs[i]['name']: partition.membership[i] for i in range(len(g.vs))}

# Display the result
for node, community in community_labels.items():
    print(f"Node {node} is in community {community}")


In [60]:
import json
import igraph as ig
from leidenalg import find_partition, ModularityVertexPartition

def load_json(file_path):
    """Load JSON data from a file."""
    with open(file_path, 'r') as file:
        return json.load(file)

def extract_entities_and_relationships(data):
    """Extract entities and relationships from JSON data."""
    entities = {}
    edges = []

    for graph in data:
        if graph:
            # Create nodes for each entity
            for entity in graph['entities']:
                if entity:
                    entities[entity['id']] = entity
        
        # Create edges for each relationship
        for relationship in graph['relationships']:
            source, relation, target = relationship.split('[')[0], relationship.split('[')[1][:-1], relationship.split(']')[1]
            edges.append((source, target))
    
    return entities, edges

def create_graph(entities, edges):
    """Create a graph using igraph from entities and edges."""
    g = ig.Graph()
    g.add_vertices(list(entities.keys()))
    g.add_edges(edges)
    return g

def apply_leiden_algorithm(graph):
    """Apply the Leiden algorithm to detect communities."""
    partition = find_partition(graph, ModularityVertexPartition)
    community_labels = {graph.vs[i]['name']: partition.membership[i] for i in range(len(graph.vs))}
    return community_labels

def generate_cypher_statements(entities, edges, community_labels):
    """Generate Cypher statements to ingest data into Neo4j."""
    cypher_statements = []

    # Create nodes
    for entity_id, entity_data in entities.items():
        properties = ", ".join([f"{key}: '{value}'" for key, value in entity_data.items() if value])
        community_label = community_labels[entity_id]
        cypher_statements.append(f"MERGE (n:{entity_data['label']} {{id: '{entity_id}'}}) SET n += {{{properties}}}, n.community = {community_label};")

    # Create relationships
    for source, target in edges:
        relationship_type = next((r.split('[')[1][:-1] for r in data[0]['relationships'] if r.startswith(source)), None)
        if relationship_type:
            cypher_statements.append(f"MATCH (a {{id: '{source}'}}), (b {{id: '{target}'}}) MERGE (a)-[:{relationship_type}]->(b);")

    return cypher_statements

def generate_cypher_statements(data):

    # Extract entities and relationships
    entities, edges = extract_entities_and_relationships(data)

    # Create graph and apply Leiden algorithm
    graph = create_graph(entities, edges)
    community_labels = apply_leiden_algorithm(graph)

    # Generate Cypher statements
    cypher_statements = generate_cypher_statements(entities, edges, community_labels)

    # Print Cypher statements
    for statement in cypher_statements:
        print(statement)

generate_cypher_statements("files/entities_and_relations_2.json")


TypeError: string indices must be integers, not 'str'

In [41]:
import igraph as ig
import leidenalg as la
import json

# Normalize IDs to ensure consistent matching
def normalize_id(id):
    return id.replace("-", "").replace("_", "").strip().lower()

# Step 1: Initialize entities with a default community
def initialize_entities_with_default_community(entities):
    # Assign a default community (e.g., -1) to all entities
    for entity in entities:
        entity['community'] = -1  # Default community
        # Normalize the entity ID for consistent matching
        entity['id'] = normalize_id(entity['id'])
    return entities

# Step 2: Create the graph from entities and relationships
def create_graph_for_community_detection(entities, relationships):
    G = ig.Graph(directed=False)

    # Extract entity IDs, clean them, and add them as vertices
    entity_ids = [entity['id'] for entity in entities]  # Already normalized in initialization
    G.add_vertices(entity_ids)

    # Add edges to the graph based on relationships
    for rs in relationships:
        rs = rs.strip()
        if "[" in rs and "]" in rs:  # Handle square bracket format
            try:
                src_id, rest = rs.split("[", 1)
                rs_type, tgt_id = rest.split("]", 1)
            except ValueError:
                print(f"Invalid relationship format: '{rs}'. Skipping.")
                continue
        elif rs.count("|") == 2:  # Handle pipe format
            try:
                src_id, rs_type, tgt_id = rs.split("|")
            except ValueError:
                print(f"Invalid relationship format: '{rs}'. Skipping.")
                continue
        else:
            print(f"Invalid relationship format: '{rs}'. Skipping.")
            continue

        # Normalize and format IDs
        src_id = normalize_id(src_id)
        tgt_id = normalize_id(tgt_id)

        # Only add edges if both source and target IDs are in the graph
        if src_id in entity_ids and tgt_id in entity_ids:
            G.add_edge(src_id, tgt_id)
        else:
            print(f"IDs {src_id} or {tgt_id} not found in entity list. Skipping.")

    return G

# Step 3: Apply Leiden algorithm to detect real communities
def apply_leiden_algorithm(G, entities):
    try:
        # Apply Leiden algorithm to detect real communities
        partition = la.find_partition(G, la.ModularityVertexPartition)
        for idx, entity in enumerate(entities):
            entity['community'] = partition.membership[idx]
    except Exception as e:
        print(f"Error while applying the Leiden algorithm: {e}")
        raise

    return entities

# Step 4: Generate Cypher queries using annotated entities
def generate_cypher_with_community(entities_relationships):
    e_statements = []
    r_statements = []
    e_label_map = {}

    for obj in entities_relationships:
        for entity in obj['entities']:
            label = entity['label']
            id = entity['id']

            # Ensure each entity has a community (already assigned default or real)
            properties = {k: v for k, v in entity.items() if k not in ['label', 'id']}
            properties['community'] = entity['community']

            cypher = f"MERGE (n:{label} {{id: '{id}'}})"
            if properties:
                props_string = ', '.join([f'n.{key} = "{val}"' for key, val in properties.items()])
                cypher += f" ON CREATE SET {props_string}"
            e_statements.append(cypher)
            e_label_map[id] = label

        # Generate Cypher queries for relationships
        for rs in obj['relationships']:
            rs = rs.strip()
            if "[" in rs and "]" in rs:
                try:
                    src_id, rest = rs.split("[", 1)
                    rs_type, tgt_id = rest.split("]", 1)
                except ValueError:
                    print(f"Invalid relationship format: '{rs}'. Skipping.")
                    continue
            elif rs.count("|") == 2:
                try:
                    src_id, rs_type, tgt_id = rs.split("|")
                except ValueError:
                    print(f"Invalid relationship format: '{rs}'. Skipping.")
                    continue
            else:
                print(f"Invalid relationship format: '{rs}'. Skipping.")
                continue

            # Normalize and format IDs
            src_id = normalize_id(src_id)
            tgt_id = normalize_id(tgt_id)
            src_label = e_label_map.get(src_id)
            tgt_label = e_label_map.get(tgt_id)

            # Generate Cypher query for valid relationships
            if src_label and tgt_label:
                cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
                r_statements.append(cypher)
            else:
                print(f"Missing labels for entities in relationship '{rs}'. Skipping.")

    # Save Cypher queries to a file
    with open("cyphers_with_community.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))
    return e_statements + r_statements

# Step 5: Process entities and relationships, then generate Cypher queries
def community_generation(data_list):
    entities_relationships = []

    # Iterate over the list of data
    for obj in data_list:
        entities = obj['entities']
        relationships = obj['relationships']

        # Step 1: Initialize entities with a default community
        entities = initialize_entities_with_default_community(entities)

        # Step 2: Create the graph
        G = create_graph_for_community_detection(entities, relationships)

        # Step 3: Apply the Leiden algorithm to detect real communities
        entities_with_communities = apply_leiden_algorithm(G, entities)

        # Update entities with real communities
        obj['entities'] = entities_with_communities

        # Append the processed object back to the list
        entities_relationships.append(obj)

    # Generate Cypher queries after all entities have their communities assigned
    cypher_statements = generate_cypher_with_community(entities_relationships)

    return cypher_statements

# Step 6: Execute the generated Cypher queries
def ingestion_pipeline_with_communities(cypher_statements):
    for stat in cypher_statements:
        try:
            gd.execute_query(stat)
        except Exception as e:
            with open("failed_statements.txt", "a") as f:
                f.write(f"{stat} - Exception: {e}\n")

# Step 7: Read input file and start pipeline
with open("files/entities_and_relations_2.json", "r") as f:
    json_obj = json.load(f)

# Start the process using the updated code
cypher_statements = community_generation(json_obj)



IDs urbangrowthseminar or daniel m. sturm not found in entity list. Skipping.
IDs uschinaseminarseries or phelanunitedstatescentre not found in entity list. Skipping.
IDs africatalks or firozlaljiinstituteforafrica not found in entity list. Skipping.
IDs tuitionfeepolicy202122 or financedivision not found in entity list. Skipping.
IDs tuitionfeepolicy202223 or financedivision not found in entity list. Skipping.
IDs tuitionfeepolicy202324 or financedivision not found in entity list. Skipping.
IDs tuitionfeepolicy202425 or financedivision not found in entity list. Skipping.
IDs creditmanagementpolicy202122 or financedivision not found in entity list. Skipping.
IDs creditmanagementpolicy202223 or financedivision not found in entity list. Skipping.
IDs creditmanagementpolicy202324 or financedivision not found in entity list. Skipping.
IDs creditmanagementpolicy202425 or financedivision not found in entity list. Skipping.
IDs studentcompensationandrefundpolicy or financedivision not found i

IDs bingchunmeng or phdacademyoffice not found in entity list. Skipping.
IDs petermills or phdacademy not found in entity list. Skipping.
IDs mariachristinavogkli or phdacademy not found in entity list. Skipping.
IDs freyagrisoni or phdacademy not found in entity list. Skipping.
IDs liudmilapliner or phdacademy not found in entity list. Skipping.
IDs shumabegum or phdacademy not found in entity list. Skipping.
IDs loraineevans or phdacademy not found in entity list. Skipping.
IDs faithlangley or phdacademy not found in entity list. Skipping.
IDs kirstyrawlings or phdacademy not found in entity list. Skipping.
IDs davidtiedemann or phdacademy not found in entity list. Skipping.
IDs joemacdonald or phdacademy not found in entity list. Skipping.
IDs theallocationofauthorityinorganizations or profjohnvanreenen not found in entity list. Skipping.
IDs optimalautomaticstabilizers or profricardoreis not found in entity list. Skipping.
IDs areplytocampbellandmau or profjohnvanreenen not found i

TypeError: 'NoneType' object is not subscriptable

## 4. Neo4j Cypher Generation without community levels

In [ ]:
# Function takes json-obejcts of entities and relationships and generates a cypher query for creating those entities
# Does so without use of Leiden clustering

import re

def generate_cypher(json_obj):
    e_statements = []
    r_statements = []
    e_label_map = {}

    # Loop through our json object
    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for file {i+1} of {len(json_obj)}")

        # Skip if obj is None
        if obj is None:
            print(f"Skipping null object at index {i}")
            continue
        
        # Process entities
        for entity in obj.get("entities", []):
            # Skip if entity is None
            if entity is None:
                continue
            label = entity.get("label")
            id = entity.get("id")
            
            # Skip if label or id is None
            if label is None or id is None:
                continue
            
            id = id.replace("-", "").replace("_", "")
            properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}

            cypher = f'MERGE (n:{label} {{id: "{id}"}})'
            if properties:
                props_str = ", ".join([f'n.{key} = "{val}"' for key, val in properties.items()])
                cypher += f" ON CREATE SET {props_str}"
            
            e_statements.append(cypher)
            e_label_map[id] = label  # Store the label in the map

        # Process relationships
        for rs in obj.get("relationships", []):
            # Skip if relationship is None
            if rs is None:
                continue
            
            # Use regular expression to extract src_id, relationship type, and tgt_id
            match = re.match(r"(.+)\[(.+)\](.+)", rs)
            if not match:
                print(f"Skipping invalid relationship format: {rs}")
                continue

            # Extract the components from the regex match
            src_id, rs_type, tgt_id = match.groups()
            
            # Clean the IDs
            src_id = src_id.replace("-", "").replace("_", "")
            tgt_id = tgt_id.replace("-", "").replace("_", "")

            # Get the source and target labels from the map
            src_label = e_label_map.get(src_id)
            tgt_label = e_label_map.get(tgt_id)

            # Skip if either source or target label is missing
            if not src_label or not tgt_label:
                print(f"Skipping relationship due to missing entity label for {src_id} or {tgt_id}")
                continue

            # Generate the relationship cypher statement
            cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
            r_statements.append(cypher)

    return e_statements + r_statements

    # Write the generated cypher statements to a file
    with open("cyphers.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements

def ingestion_pipeline(cypher_statements):
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            gds.execute_query(stmt)
        except Exception as e:
            # Log the failed statement to a file
            with open("files/failed_statements.txt", "a") as f:  # Use 'a' to append to the file instead of overwriting it
                f.write(f"{stmt} - Exception: {e}\n")

with open("files/entities_and_relations_2.json", "r") as f:
    json_obj = json.load(f)

cypher = generate_cypher(json_obj)

In [ ]:
# Function takes json-obejcts of entities and relationships and generates a cypher query for creating those entities
# Does so without use of Leiden clustering

import re

def generate_cypher(json_obj):
    e_statements = []
    r_statements = []
    e_label_map = {}

    # Loop through our json object
    for i, obj in enumerate(json_obj):
        print(f"Generating cypher for file {i+1} of {len(json_obj)}")

        # Skip if obj is None
        if obj is None:
            print(f"Skipping null object at index {i}")
            continue
        
        # Process entities
        for entity in obj.get("entities", []):
            # Skip if entity is None
            if entity is None:
                continue
            label = entity.get("label")
            id = entity.get("id")
            
            # Skip if label or id is None
            if label is None or id is None:
                continue
            
            id = id.replace("-", "").replace("_", "")
            properties = {k: v for k, v in entity.items() if k not in ["label", "id"]}

            cypher = f'MERGE (n:{label} {{id: "{id}"}})'
            if properties:
                props_str = ", ".join([f'n.{key} = "{val}"' for key, val in properties.items()])
                cypher += f" ON CREATE SET {props_str}"
            
            e_statements.append(cypher)
            e_label_map[id] = label  # Store the label in the map

        # Process relationships
        for rs in obj.get("relationships", []):
            # Skip if relationship is None
            if rs is None:
                continue
            
            # Use regular expression to extract src_id, relationship type, and tgt_id
            match = re.match(r"(.+)\[(.+)\](.+)", rs)
            if not match:
                print(f"Skipping invalid relationship format: {rs}")
                continue

            # Extract the components from the regex match
            src_id, rs_type, tgt_id = match.groups()
            
            # Clean the IDs
            src_id = src_id.replace("-", "").replace("_", "")
            tgt_id = tgt_id.replace("-", "").replace("_", "")

            # Get the source and target labels from the map
            src_label = e_label_map.get(src_id)
            tgt_label = e_label_map.get(tgt_id)

            # Skip if either source or target label is missing
            if not src_label or not tgt_label:
                print(f"Skipping relationship due to missing entity label for {src_id} or {tgt_id}")
                continue

            # Generate the relationship cypher statement
            cypher = f'MERGE (a:{src_label} {{id: "{src_id}"}}) MERGE (b:{tgt_label} {{id: "{tgt_id}"}}) MERGE (a)-[:{rs_type}]->(b)'
            r_statements.append(cypher)

    return e_statements + r_statements

    # Write the generated cypher statements to a file
    with open("cyphers.txt", "w") as outfile:
        outfile.write("\n".join(e_statements + r_statements))

    return e_statements + r_statements

def ingestion_pipeline(cypher_statements):
    for i, stmt in enumerate(cypher_statements):
        print(f"Executing cypher statement {i+1} of {len(cypher_statements)}")
        try:
            gds.execute_query(stmt)
        except Exception as e:
            # Log the failed statement to a file
            with open("files/failed_statements.txt", "a") as f:  # Use 'a' to append to the file instead of overwriting it
                f.write(f"{stmt} - Exception: {e}\n")

with open("files/entities_and_relations_2.json", "r") as f:
    json_obj = json.load(f)

cypher = generate_cypher(json_obj)

In [17]:
ingestion_pipeline(cypher)

Executing cypher statement 1 of 36488
Executing cypher statement 2 of 36488
Executing cypher statement 3 of 36488
Executing cypher statement 4 of 36488
Executing cypher statement 5 of 36488
Executing cypher statement 6 of 36488
Executing cypher statement 7 of 36488
Executing cypher statement 8 of 36488
Executing cypher statement 9 of 36488
Executing cypher statement 10 of 36488
Executing cypher statement 11 of 36488
Executing cypher statement 12 of 36488
Executing cypher statement 13 of 36488
Executing cypher statement 14 of 36488
Executing cypher statement 15 of 36488
Executing cypher statement 16 of 36488
Executing cypher statement 17 of 36488
Executing cypher statement 18 of 36488
Executing cypher statement 19 of 36488
Executing cypher statement 20 of 36488
Executing cypher statement 21 of 36488
Executing cypher statement 22 of 36488
Executing cypher statement 23 of 36488
Executing cypher statement 24 of 36488
Executing cypher statement 25 of 36488
Executing cypher statement 26 of 3